In [14]:
# 지역명 - 지역코드 매핑 함수
def get_lawd_cd(region_name):
    region_map = {
        "서울 강남구": "11680",
        "서울 서초구": "11650",
        "서울 송파구": "11710",
        "서울 강동구": "11740",
        "서울 마포구": "11440",
        "서울 관악구": "11620"
    }
    code = region_map.get(region_name)
    if code:
        print(f"[지역 코드] {region_name} -> {code}")
        return code
    else:
        print(f"[오류] '{region_name}'에 대한 코드가 없습니다")
        return None

In [15]:
# 실거래가 수집 함수
import requests
import xml.etree.ElementTree as ET

def get_apt_trade_info(lawd_cd, deal_ym, service_key):
    print(f"[조회] 지역코드={lawd_cd}, 조회월={deal_ym}")
    url = "https://apis.data.go.kr/1613000/RTMSDataSvc/getRTMSDataSvcAptTrade"
    params = {
        "serviceKey": service_key,
        "LAWD_CD": lawd_cd,
        "DEAL_YMD": deal_ym
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        res.raise_for_status()
        root = ET.fromstring(res.content)
        items = root.findall(".//item")
        print(f"[성공] 거래 건수: {len(items)}")
        return items
    except Exception as e:
        print(f"[오류] 거래 정보 조회 실패: {e}")
        return []


In [16]:
# DataFrame 구성 및 저장
import pandas as pd

def parse_items_to_dataframe(items, region_name, deal_ym):
    data = []
    for item in items:
        data.append({
            "지역명": region_name,
            "거래일자": f"{item.findtext('년')}-{item.findtext('월')}-{item.findtext('일')}",
            "아파트": item.findtext("아파트"),
            "거래금액(만원)": item.findtext("거래금액").strip().replace(",", ""),
            "건축년도": item.findtext("건축년도"),
            "전용면적": item.findtext("전용면적"),
            "층": item.findtext("층"),
            "지번": item.findtext("지번"),
            "법정동": item.findtext("법정동")
        })
    df = pd.DataFrame(data)
    filename = f"{region_name.replace(' ', '_')}_아파트실거래_{deal_ym}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"[CSV 저장 완료] → {filename}")
    return df


In [23]:
# 실행 예시
SERVICE_KEY = '69736670776c6a68343455724b7853'
region_name = "서울 강남구"
deal_ym = "202405"

lawd_cd = get_lawd_cd(region_name)
if lawd_cd:
    items = get_apt_trade_info(lawd_cd, deal_ym, SERVICE_KEY)
    if items:
        df = parse_items_to_dataframe(items, region_name, deal_ym)
        print(df.head())


[지역 코드] 서울 강남구 -> 11680
[조회] 지역코드=11680, 조회월=202405
[오류] 거래 정보 조회 실패: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /1613000/RTMSDataSvc/getRTMSDataSvcAptTrade?serviceKey=69736670776c6a68343455724b7853&LAWD_CD=11680&DEAL_YMD=202405 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1016)')))


In [27]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# ✅ 인코딩된 키를 URL에 직접 포함 (params로 보내지 않음)
SERVICE_KEY = "aXIig7X%2FAH2qZfj%2FWPpol5Lmwbj2z3QywRQqJwchXiEqWSVmydOS5TzxxLAiTyb4ySXJrbTMB2QLF3hS0Kep6g%3D%3D"
LAWD_CD = '11680'
DEAL_YMD = '202405'

# 🔧 올바른 REST API URL
url = "https://apis.data.go.kr/1611000/RTMSDataSvc/getRTMSDataSvcAptTrade"

params = {
    "LAWD_CD": LAWD_CD,
    "DEAL_YMD": DEAL_YMD
}

try:
    # ✅ 인증키는 이미 URL에 포함돼 있으므로, params에 다시 넣지 마세요!
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()
    print(f"응답 성공: {response.status_code}")
    
    # ✅ XML 파싱
    root = ET.fromstring(response.content)
    items = root.findall(".//item")
    print(f"조회된 거래 건수: {len(items)}")

    # ✅ 예시 출력
    for item in items[:5]:
        print(item.findtext("아파트"), item.findtext("거래금액").strip())

except Exception as e:
    print("[오류 발생]", e)


[오류 발생] HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /1611000/RTMSDataSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202405 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1016)')))


In [33]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# 국토부 인코딩된 인증키(Encoding Key)
SERVICE_KEY = 'aXIig7X%2FAH2qZfj%2FWPpol5Lmwbj2z3QywRQqJwchXiEqWSVmydOS5TzxxLAiTyb4ySXJrbTMB2QLF3hS0Kep6g%3D%3D'

# 조회할 지역 코드와 년월
LAWD_CD = '11680'
DEAL_YMD = '202405'

# url 구성 확인
url = f"https://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTrade"
params = {
    "serviceKey": SERVICE_KEY,
    "LAWD_CD": LAWD_CD,
    "DEAL_YMD": DEAL_YMD
}

try:
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()  # 예외 발생
    print(f'응답 성공: {response.status_code}')
    print("📄 응답 미리보기:", response.text[:300])  # 일부만 출력해서 구조 확인

    # XML 파싱
    root = ET.fromstring(response.content)
    items = root.findall(".//item")
    print(f"🔍 조회된 거래 건수: {len(items)}")

    # 결과 출력
    for item in items[:5]:
        print("📌 아파트:", item.findtext("아파트"), "| 거래금액:", item.findtext("거래금액").strip())

except Exception as e:
    print("❌ 오류 발생:", e)

❌ 오류 발생: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTrade?serviceKey=aXIig7X%252FAH2qZfj%252FWPpol5Lmwbj2z3QywRQqJwchXiEqWSVmydOS5TzxxLAiTyb4ySXJrbTMB2QLF3hS0Kep6g%253D%253D&LAWD_CD=11680&DEAL_YMD=202405 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1016)')))


In [ ]:
def parse_items_to_dataframe(items, region_name, deal_ym):
    data = []
    for item in items:
        data.append({
            "지역명": region_name,
            "거래일자": f"{item.findtext('년')}-{item.findtext('월')}-{item.findtext('일')}",
            "아파트": item.findtext("아파트"),
            "거래금액(만원)": item.findtext("거래금액").strip().replace(",", ""),
            "건축년도": item.findtext("건축년도"),
            "전용면적": item.findtext("전용면적"),
            "층": item.findtext("층"),
            "지번": item.findtext("지번"),
            "법정동": item.findtext("법정동")
        })
    df = pd.DataFrame(data)
    filename = f"{region_name.replace(' ', '_')}_아파트실거래_{deal_ym}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"[CSV 저장 완료] → {filename}")
    return df
